In [1]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
kia = np.load(('car project/kia.npy'),allow_pickle='TRUE').item()

In [3]:
kia.keys()

dict_keys(['니로 EV', '니로 플러스', 'EV6', 'EV6 GT', '봉고Ⅲ EV', '봉고III 탑차/윙바디 EV', '봉고III 파워게이트 EV', '모닝', 'K3', 'K3 GT', 'K5', 'K8', '기아 레이', '스팅어', 'K9', '셀토스', '니로', '스포티지', '쏘렌토', '모하비', '카니발', '카니발 하이리무진', 'K8택시', '봉고Ⅲ 트럭', '봉고III 탑차/윙바디/워크스루밴', '봉고III 덤프', '봉고III 파워게이트', '봉고III 와이드트럭/홈로리/활어수송차', '그랜버드'])

In [4]:
def classify_car(img,carList):  # np.array 형태의 이미지를 받아서 차량으로 분류되면 리스트에 저장하는 함수
    w = 'yolov3.weights'  # 본인 컴퓨터 경로로 설정
    c = 'yolov3.cfg'
    try:
        yolo_model = cv2.dnn.readNet(w,c)  # 기존에 학습된 yolo 모델 불러오기
        blob = cv2.dnn.blobFromImage(img,1.0/255,(128,128),(0,0,0),swapRB=True,crop=False)  # 이미지를 blob형식으로 변경
        yolo_model.setInput(blob)  # blob형식으로 변경한 이미지를 input으로 yolo 모델에 넣어준다
    
        # yolo 모델에 사용된 layer 중 yolo에 해당하는 layer만 가져오기
        layer_names = yolo_model.getLayerNames()
        out_layers = [layer_names[i-1] for i in yolo_model.getUnconnectedOutLayers()]   
        outputs = yolo_model.forward(out_layers)

        class_ids, confidences, boxes = [],[],[]
        height, width, channels = img.shape

        # 확률이 0.5를 넘는 바운딩 박스를 생성하고 모음
        for output in outputs:
          for vec85 in output:
            scores = vec85[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # 신뢰도가 50% 초과인 경우만 취급
              centerx,centery = int(vec85[0]*width), int(vec85[1]*height)
              # 영상 크기로 변환
              w,h = int(vec85[2]*width), int(vec85[3]*height)
              x,y = int(centerx - w / 2), int(centery - h / 2)
              boxes.append([x,y,w,h])
              confidences.append(float(confidence))
              class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.4)  # 분류된 것 중에 최대인 것만 남김

        classes = []
        with open('coco.names.txt', 'r') as f:
          classes = [line.strip() for line in f.readlines()]

        for i in indexes:
            if classes[class_ids[i]] == 'car':  # 분류명이 car일 때 리스트에 저장
                x,y,w,h = boxes[i]
                if all([w >= width * 0.5, h >= height * 0.5]):
                    img = cv2.resize(img, (128,128))  # 이미지 사이즈 (128,128)로 변경
                    carList.append(img)
                    break
    except:
        return

In [5]:
kiakia = {}

In [6]:
want = ['니로 EV', 'EV6', 'K5', '카니발', '스포티지', '스팅어', '셀토스', '쏘렌토', '모하비', '모닝', 'K9', 'K3']

In [7]:
for i in want:
    ccc = []
    for j in range(len(kia[i])):
        classify_car(kia[i][j],ccc)
    kiakia[i] = ccc

In [8]:
kiakia.keys()

dict_keys(['니로 EV', 'EV6', 'K5', '카니발', '스포티지', '스팅어', '셀토스', '쏘렌토', '모하비', '모닝', 'K9', 'K3'])

In [9]:
np.save('car project/kiakia.npy', kiakia)